In [0]:
!git clone https://github.com/rogersstuart/DeepDreamAnimation.git

from google.colab import drive
drive.mount('/content/drive')

## Caffe Prerpare

In [0]:
!nvidia-smi
!nvcc -V

if the output is
>**Cuda compilation tools, release 10.0, V10.0.130**

then we needn't run below code.


In [0]:
!apt  --purge remove "cublas*" "cuda*"
!apt  --purge remove "nvidia*" 
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub && sudo apt update
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
#!apt update
!apt install -y cuda=10.0.130-1

# Install CuDNN 7 and NCCL 2
!wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!dpkg -i nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb

#!apt update
!apt install -y libcudnn7 libcudnn7-dev libnccl2 libc-ares-dev

!apt autoremove
#!apt upgrade

# Link libraries to standard locations
!mkdir -p /usr/local/cuda-10.0/nccl/lib
!ln -s /usr/lib/x86_64-linux-gnu/libnccl.so.2 /usr/local/cuda/nccl/lib/
!ln -s /usr/lib/x86_64-linux-gnu/libcudnn.so.7 /usr/local/cuda-10.0/lib64/

get GPU information

In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

### Build Caffe

In [0]:
 !apt install python3-opencv
 !apt-get install libopenblas-dev # OpenBLAS
 # Other dependencies
!apt-get install libprotobuf-dev libleveldb-dev libsnappy-dev libopencv-dev libhdf5-serial-dev protobuf-compiler
!apt-get install — no-install-recommends libboost-all-dev
!apt-get install libgflags-dev libgoogle-glog-dev liblmdb-dev
!pip3 install protobuf
!apt-get install the python3-dev

#### clone caffe project
>rectified version

In [0]:
import os
os.chdir('/content')
!rm -rf caffe
!git clone https://github.com/TimeIsTheChoice/caffe.git
!cd caffe

#### build caffe

>build pycaffe

In [0]:
import os
os.chdir('/content/caffe')
!make all -j$(nproc --all) # 4 represents number of CPU Cores
!make pycaffe -j$(nproc --all) # 4 represents number of CPU Cores

In [0]:
!export PYTHONPATH=~/caffe/python:$PYTHONPATH

#### import caffe

In [0]:
caffe_root = '/content/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)

import sys
sys.path.insert(0, caffe_root + 'python')

### GPU environment information



> make sure execute under gpu model



In [0]:
#' ' means CPU whereas '/device:G:0' means GPU
import tensorflow as tf
tf.test.gpu_device_name()

>check GPU memory utilization

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

In [0]:
!rm -rf /content/myvideo
!mkdir /content/myvideo
!python /content/DeepDreamAnimation/dreamer.py --input "/content/drive/My\ Drive/rec_2.mp4" --output /content/myvideo --extract 1

In [0]:
!python /content/caffe/scripts/download_model_binary.py /content/caffe/models/bvlc_googlenet

In [0]:

#!python /content/dreamer/dreamer.py --input /content/myvideo --output /content/myvideo/frames
!python /content/DeepDreamAnimation/dreamer.py --input /content/myvideo --output /content/drive/My\ Drive/dream_out/frames_25 --octaves 4 --octavescale 2 --iterations 20 --jitter 1 --zoom 1 --stepsize 1 --blend 0 --flow 1 --layers inception_5a/output --gpu 1

In [0]:
!rm -rf /content/myvideo/frames 